## POS Tagging - Lexicon and Rule Based Taggers

Let's look at the two most basic tagging techniques - lexicon based (or unigram) and rule-based. 

In this guided exercise, you will explore the WSJ (wall street journal) POS-tagged corpus that comes with NLTK and build a lexicon and rule-based tagger using this corpus as the tarining data. 

This exercise is divided into the following sections:
1. Reading and understanding the tagged dataset
2. Exploratory analysis

### 1. Reading and understanding the tagged dataset

In [3]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import math
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [4]:
# reading the Treebank tagged sentences
wsj = list(nltk.corpus.treebank.tagged_sents())

In [5]:
# samples: Each sentence is a list of (word, pos) tuples
wsj[:3]

[[('Pierre', 'NNP'),
  ('Vinken', 'NNP'),
  (',', ','),
  ('61', 'CD'),
  ('years', 'NNS'),
  ('old', 'JJ'),
  (',', ','),
  ('will', 'MD'),
  ('join', 'VB'),
  ('the', 'DT'),
  ('board', 'NN'),
  ('as', 'IN'),
  ('a', 'DT'),
  ('nonexecutive', 'JJ'),
  ('director', 'NN'),
  ('Nov.', 'NNP'),
  ('29', 'CD'),
  ('.', '.')],
 [('Mr.', 'NNP'),
  ('Vinken', 'NNP'),
  ('is', 'VBZ'),
  ('chairman', 'NN'),
  ('of', 'IN'),
  ('Elsevier', 'NNP'),
  ('N.V.', 'NNP'),
  (',', ','),
  ('the', 'DT'),
  ('Dutch', 'NNP'),
  ('publishing', 'VBG'),
  ('group', 'NN'),
  ('.', '.')],
 [('Rudolph', 'NNP'),
  ('Agnew', 'NNP'),
  (',', ','),
  ('55', 'CD'),
  ('years', 'NNS'),
  ('old', 'JJ'),
  ('and', 'CC'),
  ('former', 'JJ'),
  ('chairman', 'NN'),
  ('of', 'IN'),
  ('Consolidated', 'NNP'),
  ('Gold', 'NNP'),
  ('Fields', 'NNP'),
  ('PLC', 'NNP'),
  (',', ','),
  ('was', 'VBD'),
  ('named', 'VBN'),
  ('*-1', '-NONE-'),
  ('a', 'DT'),
  ('nonexecutive', 'JJ'),
  ('director', 'NN'),
  ('of', 'IN'),
  ('this'

In the list mentioned above, each element of the list is a sentence. Also, note that each sentence ends with a full stop '.' whose POS tag is also a '.'. Thus, the POS tag '.' demarcates the end of a sentence.

Also, we do not need the corpus to be segmented into sentences, but can rather use a list of (word, tag) tuples. Let's convert the list into a (word, tag) tuple.

In [6]:
# converting the list of sents to a list of (word, pos tag) tuples
tagged_words = [tup for sent in wsj for tup in sent]
print(len(tagged_words))
tagged_words[:10]

100676


[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT')]

We now have a list of about 100676 (word, tag) tuples. Let's now do some exploratory analyses.

### 2. Exploratory Analysis

Let's now conduct some basic exploratory analysis to understand the tagged corpus. To start with, let's ask some simple questions:
1. How many unique tags are there in the corpus? 
2. Which is the most frequent tag in the corpus?
3. Which tag is most commonly assigned to the following words:
    - "bank"
    - "executive"


In [8]:
# question 1: Find the number of unique POS tags in the corpus
# you can use the set() function on the list of tags to get a unique set of tags, 
# and compute its length
tags =  [i[1] for i in tagged_words]
unique_tags = set(tags)
len(unique_tags)

46

In [15]:
# question 2: Which is the most frequent tag in the corpus
# to count the frequency of elements in a list, the Counter() class from collections
# module is very useful, as shown below

from collections import Counter
tag_counts = Counter(tags)
max(list(tag_counts.values()))
tag_counts

Counter({'#': 16,
         '$': 724,
         "''": 694,
         ',': 4886,
         '-LRB-': 120,
         '-NONE-': 6592,
         '-RRB-': 126,
         '.': 3874,
         ':': 563,
         'CC': 2265,
         'CD': 3546,
         'DT': 8165,
         'EX': 88,
         'FW': 4,
         'IN': 9857,
         'JJ': 5834,
         'JJR': 381,
         'JJS': 182,
         'LS': 13,
         'MD': 927,
         'NN': 13166,
         'NNP': 9410,
         'NNPS': 244,
         'NNS': 6047,
         'PDT': 27,
         'POS': 824,
         'PRP': 1716,
         'PRP$': 766,
         'RB': 2822,
         'RBR': 136,
         'RBS': 35,
         'RP': 216,
         'SYM': 1,
         'TO': 2179,
         'UH': 3,
         'VB': 2554,
         'VBD': 3043,
         'VBG': 1460,
         'VBN': 2134,
         'VBP': 1321,
         'VBZ': 2125,
         'WDT': 445,
         'WP': 241,
         'WP$': 14,
         'WRB': 178,
         '``': 712})

In [16]:
# the most common tags can be seen using the most_common() method of Counter
tag_counts.most_common(5)

[('NN', 13166), ('IN', 9857), ('NNP', 9410), ('DT', 8165), ('-NONE-', 6592)]

Thus, NN is the most common tag followed by IN, NNP, DT, -NONE- etc. You can read the exhaustive list of tags using the NLTK documentation as shown below.

In [18]:
# list of POS tags in NLTK
nltk.download('tagsets')
nltk.help.upenn_tagset()

[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je j

In [62]:
# question 3: Which tag is most commonly assigned to the word w. Get the tags list that appear for word w and then use the Counter()
# Try w ='bank'
all_words = [i for i in tagged_words if i[0].lower()=='bank']
bank = Counter([i for i in tagged_words if i[0].lower() == 'bank'])
bank

Counter({('Bank', 'NNP'): 32, ('bank', 'NN'): 38})

In [25]:
# question 3: Which tag is most commonly assigned to the word w. Try 'executive' 
executive = Counter([i for i in tagged_words if i[0] == 'executive'])
executive

Counter({('executive', 'JJ'): 28, ('executive', 'NN'): 40})

### 2. Exploratory Analysis Contd.

Until now, we were looking at the frequency of tags assigned to particular words, which is the basic idea used by lexicon or unigram taggers. Let's now try observing some rules which can potentially be used for POS tagging. 

To start with, let's see if the following questions reveal something useful:

4. What fraction of words with the tag 'VBD' (verb, past tense) end with the letters 'ed'
5. What fraction of words with the tag 'VBG' (verb, present participle/gerund) end with the letters 'ing'

In [52]:
# 4. how many words with the tag 'VBD' (verb, past tense) end with 'ed'
# first get the all the words tagged as VBD
tag_counts_1 = dict(tag_counts)
past_tense_verbs = [tag_counts_1[i] for i in tag_counts_1 if i=='VBG'][0]
print(past_tense_verbs)
# subset the past tense verbs with words ending with 'ed'. (Try w.endswith('ed'))
ed_verbs = Counter([i for i in tagged_words if i[0].endswith('ing')])
tag_counts_2 = dict(ed_verbs)
print(tag_counts_2)
ed_verbs = sum([tag_counts_2[i] for i in tag_counts_2 if i[1]=='VBG'])
print(ed_verbs)
print(ed_verbs / past_tense_verbs)

1460
{('publishing', 'VBG'): 4, ('causing', 'VBG'): 5, ('using', 'VBG'): 16, ('bring', 'VB'): 9, ('talking', 'VBG'): 4, ('having', 'VBG'): 11, ('making', 'VBG'): 20, ('surviving', 'VBG'): 3, ('including', 'VBG'): 38, ('striking', 'JJ'): 2, ('finding', 'NN'): 4, ('according', 'VBG'): 34, ('remaining', 'VBG'): 9, ('cancer-causing', 'JJ'): 1, ('bearing', 'NN'): 1, ('declining', 'VBG'): 4, ('rising', 'VBG'): 11, ('during', 'IN'): 38, ('yielding', 'VBG'): 3, ('top-yielding', 'JJ'): 1, ('waiving', 'VBG'): 1, ('holding', 'VBG'): 17, ('engineering', 'NN'): 8, ('ceiling', 'NN'): 10, ('borrowing', 'NN'): 3, ('cutting', 'VBG'): 5, ('marketing', 'NN'): 14, ('manufacturing', 'VBG'): 8, ('meeting', 'NN'): 16, ('receiving', 'VBG'): 4, ('evening', 'NN'): 4, ('morning', 'NN'): 4, ('boarding', 'VBG'): 1, ('dancing', 'NN'): 1, ('duckling', 'NN'): 1, ('Knowing', 'VBG'): 1, ('standing', 'JJ'): 1, ('looking', 'VBG'): 9, ('reflecting', 'VBG'): 4, ('casting', 'VBG'): 2, ('trying', 'VBG'): 22, ('advertising', 

In [ ]:
# 5. how many words with the tag 'VBG' end with 'ing'
participle_verbs = [___ =='VBG']
ing_verbs = [___ for ___ in ___ if ___]
print(len(ing_verbs) / len(participle_verbs))
ing_verbs[:20]

## 2. Exploratory Analysis Continued

Let's now try observing some tag patterns using the fact the some tags are more likely to apper after certain other tags. For e.g. most nouns NN are usually followed by determiners DT ("The/DT constitution/NN"), adjectives JJ usually precede a noun NN (" A large/JJ building/NN"), etc. 

Try answering the following questions:
1. What fraction of adjectives JJ are followed by a noun NN? 
2. What fraction of determiners DT are followed by a noun NN?
3. What fraction of modals MD are followed by a verb VB?

In [67]:
# question: what fraction of adjectives JJ are followed by a noun NN

# create a list of all tags (without the words)
tags = [i[1] for i in tagged_words]

# create a list of JJ tags
jj_tags = [i for i in tags if i=='JJ']

# create a list of (JJ, NN) tags
jj_nn_tags = [(t, tags[index+1]) for index, t in enumerate(tags) if t=='JJ' and tags[index+1]=='NN']

print(len(jj_tags))
print(len(jj_nn_tags))
print(len(jj_nn_tags) / len(jj_tags))

5834
2611
0.4475488515598217


In [70]:
# question: what fraction of determiners DT are followed by a noun NN
dt_tags = [i for i in tags if i=='DT']
dt_nn_tags = [(t,tags[index+1]) for index, t in enumerate(tags) 
              if t=='DT' and tags[index+1]=='NN']

print(len(dt_tags))
print(len(dt_nn_tags))
print(len(dt_nn_tags) / len(dt_tags))

8165
3844
0.470789957134109


In [ ]:
# question: what fraction of modals MD are followed by a verb VB?
md_tags = [___]
md_vb_tags = [(___) for index, t in enumerate(tags) 
              if ___ and ___]

print(len(md_tags))
print(len(md_vb_tags))
print(len(md_vb_tags) / len(md_tags))